In [1]:
import pandas as pd
import numpy as np

# Mint Events

In [87]:
m1 = pd.read_csv('Raw_Data/mint_01.csv')
m2 = pd.read_csv('Raw_Data/mint_02.csv')
m3 = pd.read_csv('Raw_Data/mint_03.csv')
df_mint = pd.concat([m1,m2,m3])

In [88]:
df_mint['block_timestamp'] = pd.to_datetime(df_mint['block_timestamp'].str.replace(' UTC', ''))

In [89]:
def split_data_fields(hex_string):
    raw = hex_string[2:]
    fields = [raw[i:i+64] for i in range(0, len(raw), 64)]
    return fields if len(fields) >= 4 else [None]*4

df_mint[['owner', 'liquidity_hex', 'amount0_hex', 'amount1_hex']] = df_mint['data'].apply(lambda x: pd.Series(split_data_fields(x)))

df_mint['liquidity'] = df_mint['liquidity_hex'].apply(lambda x: int(x, 16))
df_mint['amount0'] = df_mint['amount0_hex'].apply(lambda x: int(x, 16))
df_mint['amount1'] = df_mint['amount1_hex'].apply(lambda x: int(x, 16))

df_mint['TickLower'] = df_mint['tickLower'].apply(lambda x: int(x, 16))
df_mint['TickUpper'] = df_mint['tickUpper'].apply(lambda x: int(x, 16))

In [90]:
df_mint['amount0_USDC'] = df_mint['amount0'] / 1e6

# ETH → 18 decimals
df_mint['amount1_ETH'] = df_mint['amount1'] / 1e18

df_mint['amount0_USDC'] = round(df_mint['amount0_USDC'],2)

df_mint["block_timestamp"] = df_mint["block_timestamp"].dt.date

In [91]:
df_mint1 = df_mint.drop(["lp_provider","data","tickLower","tickUpper","liquidity_hex","amount0_hex","amount1_hex","amount0",
              "amount1"],axis=1)

In [92]:
df_mint1.head(3)

,block_timestamp,transaction_hash,pool_id,event,owner,liquidity,TickLower,TickUpper,amount0_USDC,amount1_ETH
0,2025-02-08,0x33ad31b91ccbe7242ffe5b328e18f3ec60ff13297031...,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85...,000000000000000000000000d1fdf0144be118c30a53e1...,1307659228545682,197630,197640,33.42,0.0
1,2025-02-08,0x61fbf4d6116ed9c097193c279948b17e300e6aa81d0a...,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85...,000000000000000000000000d1fdf0144be118c30a53e1...,1029267614611238,197590,197600,26.36,0.0
2,2025-01-10,0x95d0602ccc591f3a3ad1a0986a847c862d455dd00f01...,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85...,00000000000000000000000033cb657e7fd57f1f2d5f39...,213308536483925,181600,209340,6078.93,1.872429


In [93]:
df_mint2 = df_mint1.groupby(['owner','block_timestamp']).agg({'transaction_hash':'count','liquidity':'sum',
                                   'amount0_USDC':'sum','amount1_ETH':'sum'}).reset_index().sort_values(by=['owner','block_timestamp'])

In [94]:
df_mint2 = df_mint2.rename(columns={
    'block_timestamp': 'mint_date',
    'transaction_hash': 'mint_counts',
    'liquidity' : 'liquidity_at_mint',
    'amount0_USDC' : 'amount0_USDC_at_mint',
    'amount1_ETH' : 'amount1_ETH_at_mint'
})

In [95]:
df_mint2.head(10)

,owner,mint_date,mint_counts,liquidity_at_mint,amount0_USDC_at_mint,amount1_ETH_at_mint
0,0000000000000000000000000000000000000dededdd16...,2025-03-10,1,36654749001414,111.85,0.03989
1,0000000000000000000000000000000000000dededdd16...,2025-03-15,2,2731462288085,8.52,0.009361
2,000000000000000000000000000000d40b595b94918a28...,2025-04-17,1,6617935707635712,119.22,0.008263
3,0000000000000000000000000d3c2261747b2079f6abd4...,2025-04-07,1,15257538482193056995,285169.52,4.257939
4,0000000000000000000000000d3c2261747b2079f6abd4...,2025-05-09,1,7701983837334182115,1284274.90,0.0
5,0000000000000000000000000d3c2261747b2079f6abd4...,2025-05-10,1,3184665955175476102,104106.62,22.306233
6,0000000000000000000000000d3c2261747b2079f6abd4...,2025-05-16,1,33635616876048665412,856747.42,0.0
7,0000000000000000000000000d3c2261747b2079f6abd4...,2025-05-23,1,37295343926660587007,901494.69,16.509753
8,0000000000000000000000001d3d5a4404d6e3fa6cd7f0...,2025-03-05,3,1336074153816572,1989.92,0.973356
9,0000000000000000000000001f2f10d1c40777ae1da742...,2024-12-29,1,21924964445783588864,595501.38,12.350361


# Burn Events

In [68]:
b1 = pd.read_csv('Raw_Data/burn_01.csv')
b2 = pd.read_csv('Raw_Data/burn_02.csv')
b3 = pd.read_csv('Raw_Data/burn_03.csv')
b4 = pd.read_csv('Raw_Data/burn_04.csv')
df_burn = pd.concat([b1,b2,b3,b4])

In [70]:
df_burn['block_timestamp'] = pd.to_datetime(df_burn['block_timestamp'].str.replace(' UTC', ''))

In [71]:

def split_data_fields(hex_string):
    if not isinstance(hex_string, str) or not hex_string.startswith("0x"):
        return ['0']*4
    raw = hex_string[2:]
    fields = [raw[i:i+64] for i in range(0, len(raw), 64)]
    return fields[:4] + ['0'] * (4 - len(fields)) if len(fields) < 4 else fields[:4]


df_burn[['owner', 'liquidity_hex', 'amount0_hex', 'amount1_hex']] = df_burn['data'].apply(
    lambda x: pd.Series(split_data_fields(x))
)

df_burn['liquidity'] = df_burn['liquidity_hex'].apply(lambda x: int(x, 16))
df_burn['amount0'] = df_burn['amount0_hex'].apply(lambda x: int(x, 16))
df_burn['amount1'] = df_burn['amount1_hex'].apply(lambda x: int(x, 16))



df_burn['TickLower'] = df_burn['tickLower'].apply(lambda x: int(x, 16))
df_burn['TickUpper'] = df_burn['tickUpper'].apply(lambda x: int(x, 16))

In [76]:
df_burn['amount0_USDC'] = df_burn['amount0'] / 1e6

# ETH → 18 decimals
df_burn['amount1_ETH'] = df_burn['amount1'] / 1e18

df_burn["block_timestamp"] = df_burn["block_timestamp"].dt.date

In [78]:
df_burn1 = df_burn.drop(["lp_provider","data","tickLower","tickUpper","liquidity_hex","amount0_hex","amount1_hex","amount0",
              "amount1"],axis=1)

In [79]:
df_burn2 = df_burn1.groupby(['owner','block_timestamp']).agg({'transaction_hash':'count','liquidity':'sum',
                                   'amount0_USDC':'sum','amount1_ETH':'sum'}).reset_index().sort_values(by=['owner','block_timestamp'])

In [81]:
df_burn2 = df_burn2.rename(columns={
    'block_timestamp': 'burn_date',
    'transaction_hash': 'burn_counts',
    'liquidity' : 'liquidity_at_burn',
    'amount0_USDC' : 'amount0_USDC_at_burn',
    'amount1_ETH' : 'amount1_ETH_at_burn'
})

In [82]:
df_burn2

,owner,burn_date,burn_counts,liquidity_at_burn,amount0_USDC_at_burn,amount1_ETH_at_burn
0,0000000000000000000000000000000000000000000000...,2024-12-29,7,0,0.0,0.0
1,0000000000000000000000000000000000000000000000...,2024-12-30,14,0,0.0,0.0
2,0000000000000000000000000000000000000000000000...,2024-12-31,18,0,0.0,0.0
3,0000000000000000000000000000000000000000000000...,2025-01-01,10,0,0.0,0.0
4,0000000000000000000000000000000000000000000000...,2025-01-02,19,0,0.0,0.0
...,...,...,...,...,...,...
29053,0000000000000000000000000000000000000000000000...,2025-02-03,1,66739447982470,0.0,0.0
29054,0000000000000000000000000000000000000000000000...,2025-02-03,1,66879668335869,0.0,0.0
29055,0000000000000000000000000000000000000000000000...,2025-02-03,1,66718884272852,0.0,0.0
29056,0000000000000000000000000000000000000000000000...,2025-06-10,1,49595517251946,13226151873752882.0,0.0


# Mint vs. Burn

In [113]:
df_lp = pd.merge(df_mint2, df_burn2, on='owner', how='outer')

In [121]:
df_lp = df_lp.sort_values(by=['owner','mint_date'])

In [122]:
df_lp.head(10)

,owner,mint_date,mint_counts,liquidity_at_mint,amount0_USDC_at_mint,amount1_ETH_at_mint,burn_date,burn_counts,liquidity_at_burn,amount0_USDC_at_burn,amount1_ETH_at_burn
1052,0000000000000000000000000000000000000000000000...,NaN,NaN,NaN,NaN,NaN,2025-04-21,51.0,0.0,0.0,0.0
1020,0000000000000000000000000000000000000000000000...,NaN,NaN,NaN,NaN,NaN,2025-03-20,32.0,0.0,0.0,0.0
1019,0000000000000000000000000000000000000000000000...,NaN,NaN,NaN,NaN,NaN,2025-03-19,36.0,0.0,0.0,0.0
1018,0000000000000000000000000000000000000000000000...,NaN,NaN,NaN,NaN,NaN,2025-03-18,23.0,0.0,0.0,0.0
1017,0000000000000000000000000000000000000000000000...,NaN,NaN,NaN,NaN,NaN,2025-03-17,42.0,0.0,0.0,0.0
1016,0000000000000000000000000000000000000000000000...,NaN,NaN,NaN,NaN,NaN,2025-03-16,35.0,0.0,0.0,0.0
1015,0000000000000000000000000000000000000000000000...,NaN,NaN,NaN,NaN,NaN,2025-03-15,20.0,0.0,0.0,0.0
1014,0000000000000000000000000000000000000000000000...,NaN,NaN,NaN,NaN,NaN,2025-03-14,26.0,0.0,0.0,0.0
1013,0000000000000000000000000000000000000000000000...,NaN,NaN,NaN,NaN,NaN,2025-03-13,40.0,0.0,0.0,0.0
1012,0000000000000000000000000000000000000000000000...,NaN,NaN,NaN,NaN,NaN,2025-03-12,41.0,0.0,0.0,0.0


In [100]:
entry_counts = df_mint2.groupby('owner').size().reset_index(name='mint_total_count')
exit_counts = df_burn2.groupby('owner').size().reset_index(name='burn_total_count')

df_counts = pd.merge(entry_counts, exit_counts, on='owner', how='outer').fillna(0)
df_counts[['mint_total_count', 'burn_total_count']] = df_counts[['mint_total_count', 'burn_total_count']].astype(int)

In [101]:
mint_amounts = df_mint2.groupby('owner')[['amount0_USDC_at_mint', 'amount1_ETH_at_mint']].sum().reset_index()
burn_amounts = df_burn2.groupby('owner')[['amount0_USDC_at_burn', 'amount1_ETH_at_burn']].sum().reset_index()

df_flows = pd.merge(mint_amounts, burn_amounts, on='owner', how='outer').fillna(0)

/var/folders/nw/7766myvd65gfypd6nd1lgsb40000gn/T/ipykernel_8492/537500526.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mint_amounts = df_mint2.groupby('owner')[['amount0_USDC_at_mint', 'amount1_ETH_at_mint']].sum().reset_index()
/var/folders/nw/7766myvd65gfypd6nd1lgsb40000gn/T/ipykernel_8492/537500526.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  burn_amounts = df_burn2.groupby('owner')[['amount0_USDC_at_burn', 'amount1_ETH_at_burn']].sum().reset_index()


In [106]:
mint_amounts

,owner,amount0_USDC_at_mint
0,0000000000000000000000000000000000000dededdd16...,1.203700e+02
1,000000000000000000000000000000d40b595b94918a28...,1.192200e+02
2,0000000000000000000000000d3c2261747b2079f6abd4...,3.431793e+06
3,0000000000000000000000001d3d5a4404d6e3fa6cd7f0...,1.989920e+03
4,0000000000000000000000001f2f10d1c40777ae1da742...,2.245129e+09
5,00000000000000000000000033cb657e7fd57f1f2d5f39...,1.774291e+05
6,000000000000000000000000381e7287eb6df7c74b5266...,4.109400e+02
7,0000000000000000000000003bfe65f217df9ad7358698...,1.099768e+06
8,0000000000000000000000004bb41165a817628992ee40...,1.650943e+06
9,00000000000000000000000051c72848c68a965f66fa7a...,7.589512e+09


In [102]:
df_flows['usdc_profit'] = df_flows['amount0_USDC_at_burn'] - df_flows['amount0_USDC_at_mint']
df_flows['eth_profit'] = df_flows['amount1_ETH_at_burn'] - df_flows['amount1_ETH_at_mint']

KeyError: 'amount0_USDC_at_burn'